In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df1=pd.read_csv('Amazon_employee_access.csv')
df2=pd.read_csv('test.csv')
test_id=df2['id']
df2=df2.drop('id',axis=1)
df=pd.concat([df1,df2],sort=False)
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1.0
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1.0
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1.0
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1.0
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1.0


In [3]:
df1=df.astype(str)
df1['target']=pd.to_numeric(df1['target'],errors='coerce')

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
#df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,1.0,117961118300
1,17183,1540,123125,118536,118539,1.0,117961118343
2,36724,14457,117884,267952,117880,1.0,118219118220
3,36135,5396,119993,240983,118322,1.0,117961118343
4,42680,5905,119569,123932,119325,1.0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91690 entries, 0 to 58920
Data columns (total 22 columns):
RESOURCE                          91690 non-null object
MGR_ID                            91690 non-null object
ROLE_DEPTNAME                     91690 non-null object
ROLE_FAMILY_DESC                  91690 non-null object
ROLE_CODE                         91690 non-null object
target                            32769 non-null float64
ROLLUP_12                         91690 non-null object
RESOURCE_MGR_ID                   91690 non-null object
RESOURCE_ROLE_DEPTNAME            91690 non-null object
RESOURCE_ROLE_FAMILY_DESC         91690 non-null object
RESOURCE_ROLE_CODE                91690 non-null object
RESOURCE_ROLLUP_12                91690 non-null object
MGR_ID_ROLE_DEPTNAME              91690 non-null object
MGR_ID_ROLE_FAMILY_DESC           91690 non-null object
MGR_ID_ROLE_CODE                  91690 non-null object
MGR_ID_ROLLUP_12                  91690 non-null o

# AST encoding with LR

In [9]:
def AST_encoding(train,test,columns,target,weight):
    tr=train.copy()
    v=test.copy()
    columns=list(columns)
   
    gm=tr[target].mean()
    for col in columns:
        gb=tr.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        tr[col+'_enc']=tr[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    tr=tr.drop(columns,axis=1)
    return tr,v

In [10]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=balanced_accuracy_score(y,pred)
    f1=f1_score(y,pred)
    return auc,acc,f1
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [11]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression



In [12]:
rem=['RESOURCE_ROLE_CODE', 'ROLE_FAMILY_DESC_ROLLUP_12', 'MGR_ID', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC',
     'MGR_ID_ROLLUP_12', 'RESOURCE_ROLE_FAMILY_DESC', 'ROLE_DEPTNAME_ROLE_CODE', 'MGR_ID_ROLE_DEPTNAME',
     'ROLE_FAMILY_DESC', 'ROLE_CODE_ROLLUP_12', 'ROLE_DEPTNAME', 'ROLE_DEPTNAME_ROLLUP_12', 'ROLLUP_12']
d1=d.drop(rem,axis=1)

In [13]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91690 entries, 0 to 58920
Data columns (total 9 columns):
RESOURCE                      91690 non-null object
ROLE_CODE                     91690 non-null object
target                        32769 non-null float64
RESOURCE_MGR_ID               91690 non-null object
RESOURCE_ROLE_DEPTNAME        91690 non-null object
RESOURCE_ROLLUP_12            91690 non-null object
MGR_ID_ROLE_FAMILY_DESC       91690 non-null object
MGR_ID_ROLE_CODE              91690 non-null object
ROLE_FAMILY_DESC_ROLE_CODE    91690 non-null object
dtypes: float64(1), object(8)
memory usage: 7.0+ MB


In [14]:
train=d1[d1.target.isnull()==False]
test=d1[d1.target.isnull()==True].drop('target',axis=1)
test.shape

(58921, 8)

In [15]:
t,v=AST_encoding(train,test,test.columns,'target',20)

x=t.drop('target',axis=1)
y=t['target']
lr=LogisticRegression(max_iter=10e7)
mod=lr.fit(x,y)
prob=mod.predict_proba(v)[:,1]
pred=mod.predict(v)
results=pd.DataFrame({'Action':prob},index=test_id)
results.to_csv('lr_1.2.csv')

In [16]:
results1=pd.DataFrame({'Action':pred},index=test_id)
#results.to_csv('lr_1.1.csv')   ##### 0.88875 AUC

In [17]:
results1['Action'].value_counts()

1.0    57951
0.0      970
Name: Action, dtype: int64

# Kfold AST encoding with Boosting

In [18]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.utils import *
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold
from sklearn.metrics import *


def kfold_AS_target(train,test,columns,target,weight,folds=10):
    
    columns=list(columns)
    kf=KFold()
    t=train
    v=test
    gm=t[target].mean()
    for col in columns:
        
        for i,j in kf.split(t):
            gb=t.loc[t.index[i]].groupby(col)[target].agg(['count','mean'])
            count=gb['count']
            mean=gb['mean']
            gb['smooth']=(count*mean + weight*gm)/(count+weight)
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(gb['smooth'])
            
        t[col+'_enc']=t[col+'_enc'].fillna(gm)
        
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
        
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [19]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc

In [20]:

train=d[d.target.isnull()==False]
test=d[d.target.isnull()==True].drop('target',axis=1)
columns=list(train.columns)
columns.remove('target')
target='target'
t,v=kfold_AS_target(train,test,columns,target,100)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [21]:
#d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91690 entries, 0 to 58920
Data columns (total 22 columns):
RESOURCE                          91690 non-null object
MGR_ID                            91690 non-null object
ROLE_DEPTNAME                     91690 non-null object
ROLE_FAMILY_DESC                  91690 non-null object
ROLE_CODE                         91690 non-null object
target                            32769 non-null float64
ROLLUP_12                         91690 non-null object
RESOURCE_MGR_ID                   91690 non-null object
RESOURCE_ROLE_DEPTNAME            91690 non-null object
RESOURCE_ROLE_FAMILY_DESC         91690 non-null object
RESOURCE_ROLE_CODE                91690 non-null object
RESOURCE_ROLLUP_12                91690 non-null object
MGR_ID_ROLE_DEPTNAME              91690 non-null object
MGR_ID_ROLE_FAMILY_DESC           91690 non-null object
MGR_ID_ROLE_CODE                  91690 non-null object
MGR_ID_ROLLUP_12                  91690 non-null o

In [22]:
rem=['ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc', 'MGR_ID_ROLE_FAMILY_DESC_enc']

xtr=t.drop([target]+rem,axis=1)
ytr=t[target]
xts=v.drop(rem,axis=1)


In [23]:
def aucroc1(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC=',auc)
    print('Accuracy=',acc)
    print('Confusion matrix:\n',confusion_matrix(y,pred))
    print('Classification report:\n',classification_report(y,pred))

In [28]:
lgb=LGBMClassifier()
mod=lgb.fit(xtr,ytr)
prob1=mod.predict_proba(xts)[:,1]
pred1=mod.predict(xts)
results2=pd.DataFrame({'Action':prob1},index=test_id)
###results2.to_csv('lgb_2.csv')           ### 0.89908 AUC

In [29]:
pip install catboost

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=aa2bf2b00fd1e17f892fdcad3116d5c195bbc841d6bf8a19f545c02efef2e6e1
  Stored in directory: C:\Users\Shashi Katteri\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.
